# Niedersachsen

Every federal state is represented by its own input directory and is processed into a NUTS level 2 directory containing a sub-folder for each discharge location. These folder names are derived from NUTS and reflect the CAMELS id. The NUTS level 2 code for Niedersachsen is `DE9`.

To pre-process the data, you need to write (at least) two functions. One should extract all metadata and condense it into a single `pandas.DataFrame`. This is used to build the folder structure and derive the ids.
The second function has to take an id, as provided by the state authorities, called `provider_id` and return a `pandas.DataFrame` with the transformed data. The dataframe needs the three columns `['date', 'q' | 'w', 'flag']`.

For easier and unified output handling, the `camelsp` package contains a context object called `Bundesland`. It takes a number of names and abbreviations to identify the correct federal state and returns an object that holds helper and save functions.

The context saves files as needed and can easily be changed to save files with different strategies, ie. fill missing data with NaN, merge data into a single file, create files for each variable or pack everything together into a netcdf.

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from typing import Union, Dict, List
from datetime import datetime as dt
from dateparser import parse
import warnings

from camelsp import Bundesland

/home/alexander/anaconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The context can also be instantiated as any regular Python class, ie. to load only the default input data path, that we will user later.

In [2]:
# the context also makes the input path available, if camelsp was install locally
BASE = Bundesland('niedersachsen').input_path
BASE

'/home/alexander/Github/camels/camelsp/input_data/NiS_Niedersachsen'

## Parse data

Niedersachen produced only one file. I guess this needs to be pivoted.

In [3]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
d_raw = dd.read_csv(os.path.join(BASE, 'exp-peg-par252.csv'), encoding='latin1', sep=';', decimal=',', parse_dates=['DATUM'], blocksize=4e6)

with ProgressBar():
    raw = d_raw.compute()

raw

[########################################] | 100% Completed | 25m 47s


,MESSSTELLE_NR,DATUM,LANGNAME,BEZEICHNUNG,KENNUNG_ID,WERT,EINHEIT
0,3183101,1985-10-01,Sudendorf,Abfluss Tagesmittelwert,NaN,0.853,m³/s
1,3183101,1985-11-01,Sudendorf,Abfluss Tagesmittelwert,NaN,0.853,m³/s
2,3183101,1985-12-01,Sudendorf,Abfluss Tagesmittelwert,NaN,0.853,m³/s
3,3183101,1985-01-13,Sudendorf,Abfluss Tagesmittelwert,NaN,0.772,m³/s
4,3183101,1987-07-09,Sudendorf,Abfluss Tagesmittelwert,NaN,0.938,m³/s
...,...,...,...,...,...,...,...
55770,3183101,1985-05-01,Sudendorf,Abfluss Tagesmittelwert,NaN,1.030,m³/s
55771,3183101,1985-06-01,Sudendorf,Abfluss Tagesmittelwert,NaN,1.030,m³/s
55772,3183101,1985-07-01,Sudendorf,Abfluss Tagesmittelwert,NaN,1.800,m³/s
55773,3183101,1985-08-01,Sudendorf,Abfluss Tagesmittelwert,NaN,1.200,m³/s


In [4]:
raw.DATUM

0       1985-10-01
1       1985-11-01
2       1985-12-01
3       1985-01-13
4       1987-07-09
           ...    
55770   1985-05-01
55771   1985-06-01
55772   1985-07-01
55773   1985-08-01
55774   1985-09-01
Name: DATUM, Length: 4195562, dtype: datetime64[ns]

In [5]:
# id column is MESSSTELLE_NR
id_column = 'MESSSTELLE_NR'

In [6]:
# How many different variables are there?
names = []
for _, df in raw.groupby(id_column):
    names.extend(df.BEZEICHNUNG.unique().tolist())
set(names)

{'Abfluss Tagesmittelwert'}

In [7]:
# total messstellen
N = len(raw.groupby(id_column))
print(f"Messstellen: {N}")

Messstellen: 282


New create a list of collected 'metadata' and the actual discharge data. 

Extract  all metadata for this federal state, without using the `Bundesland` context and then later use the context to pass extracted metadata. The Context has a function for saving *raw* metadata, that takes a `pandas.DataFrame` and needs you to identify the id column.
Here, *raw* refers to provider metadata, that has not yet been transformed into the CAMELS-de Metadata schema.

In [8]:
# result container
meta = []
data = []

# group by id
N = len(raw.groupby(id_column))

# go for it
for nr, df in tqdm(raw.groupby(id_column)):
    meta.append({
        id_column: str(nr),
        'BEZEICHNUNG': df.BEZEICHNUNG.unique().tolist(),
        'EINHEIT': df.EINHEIT.unique().tolist(),
        'LANGNAME': df.LANGNAME.unique().tolist(),
        'KENNUNG_ID': df.KENNUNG_ID.unique().tolist()
    })
    data.append(pd.DataFrame({
        'date': df.DATUM,
        'q': df.WERT,
        'flag': np.NaN
    }))

print(f"Extracted {len(data)} timeseries")
        

100%|██████████| 282/282 [00:00<00:00, 419.36it/s]

Extracted 282 timeseries


### metadata

Ok, let's get really wild. Check that the code above produced only lists of 1 unique value per group. Otherwise the metadata would change over time for the same Messstelle and that would be a problem

In [9]:
def tidy_metadata(meta: List[dict]) -> pd.DataFrame:
    pmeta = []
    for i, m in enumerate(meta):
        out = {}
        for k, v in m.items():
            if isinstance(v, list):
                if len(v) == 1:
                    out[k] = v[0]
                else:
                    warnings.warn(f"Line {i + 1}: More than one value found for {k}: [{', '.join(v)}]")
            else:
                out[k] = v
        pmeta.append(out)
    return pd.DataFrame(pmeta)


### Finally run

Now, the Q and W data can be extracted along with the metadata. The cool thing is, that all the id creation, data creation, merging and the mapping from our ids to the original ids and files is done by the context. This is helpful, as we less likely screw something up.

In [10]:
with Bundesland('Niedersachsen') as bl:
    # catch warnings
    with warnings.catch_warnings(record=True) as warns:
        # tidy the metadata
        metadata = tidy_metadata(meta)

        # save the metadata
        bl.save_raw_metadata(metadata, id_column, overwrite=True)

        # for reference, call the nuts-mapping as table
        nuts_map = bl.nuts_table
        print(nuts_map.head())
    
        # go for all ids
        for meta, df in tqdm(zip(meta, data), total=N):
            # get the id
            provider_id = meta[id_column]

            # save
            bl.save_timeseries(df, provider_id)
        
        # check if there were warnings (there are warnings)
        if len(warns) > 0:
            log_path = bl.save_warnings(warns)
            print(f"There were warnings during the processing. The log can be found at: {log_path}")


    nuts_id provider_id                              path
0  DE910000     3183101  ./DE9/DE910000/DE910000_data.csv
1  DE910010     3346103  ./DE9/DE910010/DE910010_data.csv
2  DE910020     3437108  ./DE9/DE910020/DE910020_data.csv
3  DE910030     3445100  ./DE9/DE910030/DE910030_data.csv
4  DE910040     3449100  ./DE9/DE910040/DE910040_data.csv


100%|██████████| 282/282 [00:07<00:00, 39.76it/s]
